# COVID-19 Time Series
This tutorial will visualize COVID-19 data as a time series, and along the way, show what a workflow might look like when using ahlive.

Click [here](covid_19_time_series.ipynb#final-animation) to see the full animation.

### load data

To start, we will import ahlive and abbrieviate as `ah`. Then, we open up the COVID-19 global cases dataset and display it.

`raw` is set to `True` for demonstration purposes, i.e. how to preprocess a "wide" dataset into a "tidy" dataset (this is done automatically if `raw=False`).

`verbose` is set to `True` to display the direct URL where the data is retrieved from, in addition to the source and base URL.

In [ ]:
import ahlive as ah
import pandas as pd
df = ah.tutorial.open_dataset('covid19_global_cases', raw=True, verbose=True)
display(df)

### transform data

This data is in `wide` form, but ahlive expects input data to be in "tidy" form, which is defined below:

1. Each variable must have its own column.
2. Each observation must have its own row.
3. Each type of observational unit forms a table.

Fortunately, it's easy to convert to "tidy" form using `pd.melt`.

In [ ]:
df_tidy = df.melt(
    id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'],
    var_name='Date', value_name='Cases')
display(df_tidy)

### subset data

Now, we can now use this "tidy" `pd.DataFrame` with ahlive. However, oftentimes it's best to subset and focus on a few datapoints, and also for this tutorial's simplicity sake.

In [ ]:
countries = ['US', 'China', 'New Zealand', 'United Kingdom',
             'Brazil', 'India', 'Zambia', 'Pakistan']
df_subset = df_tidy.loc[df_tidy['Country/Region'].isin(countries)]
display(df_subset)

Since this dataset was originally grouped by `Province/State`s, to further simplify and reduce crowdedness, we can group by `Country/Region` instead. Also since testing started mostly in March to April timeframe, we will begin animating in March.

In [ ]:
df_countries = df_subset.groupby(['Date', 'Country/Region'])[['Cases']].sum().reset_index()
df_countries = df_countries.loc[df_countries['Date'] >= '3/01/21']
display(df_countries)

### serialize data

Pass the preprocessed `pd.DataFrame`, the `xs` column name (what we want plotted on the x-axis), the `ys` column name (what we want plotted on the y-axis) to instantiate the `ah.DataFrame` class.

In [ ]:
ah_df = ah.DataFrame(df_countries, 'Date', 'Cases')
print(ah_df)

However, if we examine the output, we notice an abnormally large number of `state`s, or number of frames in the animation, and only 1 `item`. In other words, ahlive is unaware of the different `Country/Region`s; thus we need to pass that in as the `label`.

In [ ]:
ah_df = ah.DataFrame(df_countries, 'Date', 'Cases', label='Country/Region')
print(ah_df)

With that done, we can *almost* generate the the first animation by calling the `render` method, but before doing so, it's good to set `animate` to a list of `state`s. This limits the number of frames in the animation so we can get a preview of what the full animation looks like and ensure everything looks correct. We can also specify `fps` to prevent the animation from flashing.

In [ ]:
ah_df = ah.DataFrame(df_countries, 'Date', 'Cases', label='Country/Region',
                     animate=[20, 50, 250, 300], fps=1)
ah_df.render()

### tweak animation

By doing so, we can immediately notice that:

1. The `ylabel` is cut-off.
2. The dates are crowded.
3. COVID tests did not start until March.
4. it's hard to intuitively grasp the large numbers.
5. The legend isn't sorted by max.

To remedy this:

1. Increase the width of the figure through `figsize`.
2. Replace date `str`s with `np.datetime64` objects.
3. Slice dataframe to begin around March.
4. Scale the values by a million (`1e6`) and rerun.
5. Set `sortby='y'` in config.

Also, to reduce the size of this tutorial page, we will resample to every week.

In [ ]:
df_dts = df_countries.copy()
df_dts['Date'] = pd.to_datetime(df_dts['Date'])
df_dts = df_dts.loc[df_dts['Date'] >= '2020-03-01']
df_dts = df_dts.groupby([
    pd.Grouper(key='Date', freq='7D'), 'Country/Region'
])['Cases'].last().reset_index()
df_scale = df_dts.sort_values('Date')
df_scale['Cases'] /= 1e6

We can rerun with the newly preprocessed `pd.DataFrame`. We can also use any `str` in `animate` for a subset. It will also automatically set `fps`.

In [ ]:
ah_df = ah.DataFrame(
    df_scale, 'Date', 'Cases', label='Country/Region',
    figsize=(10, 7), animate='test'
).config('legend', sortby='y')
ah_df.render()

We now note that there are extraneous decimal points so we can use `config` to set the `format`. We may also want to add `state_labels` and `inline_labels` to see directly see the x and y values.

In [ ]:
ah_df = ah.DataFrame(
    df_scale, 'Date', 'Cases', label='Country/Region',
    state_labels='Date', inline_labels='Cases',
    figsize=(10, 7), ylabel='Confirmed Cases [million]',
    animate='test'
).config('yticks', format='%.0f').config('legend', sortby='y')
ah_df.render()

We can also set `animate` as `head_28` to animate the first 28 frames.

In [ ]:
ah_df = ah.DataFrame(
    df_scale, 'Date', 'Cases', label='Country/Region',
    state_labels='Date', inline_labels='Cases',
    figsize=(10, 7), ylabel='Confirmed Cases [million]',
    animate='head_28'
).config('yticks', format='%.0f').config('legend', sortby='y')
ah_df.render()

Since there's not much to see in the first 28 frames, we can also animate from the tail using `tail_28`.

In [ ]:
ah_df = ah.DataFrame(
    df_scale, 'Date', 'Cases', label='Country/Region',
    state_labels='Date', inline_labels='Cases',
    figsize=(10, 7), ylabel='Confirmed Cases [million]',
    animate='tail_28'
).config('yticks', format='%.0f').config('legend', sortby='y')
ah_df.render()

### cascade data

It looks decent for the most part, but there's too much going on to easily garner any insights from it and there's a lot of crowding near the bottom. We can utilize `join='cascade'` to see country by country. Also, the plot can be a bit misleading because New Zealand has a much smaller population than US so we should try normalizing by population.

In [ ]:
df_pop = ah.tutorial.open_dataset('covid19_population')[['combined_key', 'population']]
df_norm = df_scale.merge(df_pop, left_on='Country/Region', right_on='combined_key')
df_norm['Cases'] = df_norm['Cases'] * 1e6 / df_norm['population']  # previously scaled by 1 million
df_norm['Cases'] *= 1e5  # scale by per 100k people

After updating the `ylabel` to reflect the normalization, we should once again animate a few frames to see if everything looks alright.

In [ ]:
ah_df = ah.DataFrame(
    df_norm, 'Date', 'Cases', label='Country/Region',
    state_labels='Date', inline_labels='Cases',
    figsize=(10, 7), ylabel='Confirmed Cases / 100k People',
    join='cascade', animate='test'
).config('legend', sortby='y')
ah_df.render()

Looks mostly good, but the lines do not appear in a pleasing manner even with `sortby='y'` and the fixed `ylims` is not ideal. We can first sort the dataframe manually.

In [ ]:
ordering = df_norm.groupby('Country/Region').max().sort_values('Cases').index
df_sort = df_norm.copy()
df_sort['Country/Region'] = pd.Categorical(
    df_sort['Country/Region'], categories=ordering, ordered=True)
df_sort = df_sort.sort_values(['Country/Region', 'Date'])

Then set `ylims` to explore to take full advantage of the `cascade`.

In [ ]:
ah_df = ah.DataFrame(
    df_sort, 'Date', 'Cases', label='Country/Region',
    state_labels='Date', inline_labels='Cases',
    figsize=(10, 7), ylabel='Confirmed Cases / 100k People',
    join='cascade', ylims='explore',
    animate='test'
)
ah_df.render()

Let's add a title and cite the data!

In [ ]:
ah_df = ah.DataFrame(
    df_sort, 'Date', 'Cases', label='Country/Region',
    state_labels='Date', inline_labels='Cases',
    figsize=(10, 7), ylabel='Confirmed Cases / 100k People',
    join='cascade', ylims='explore',
    title='Confirmed COVID-19 Cases, normalized by population',
    note='Source: JHU CSSE COVID-19',
    animate='test'
)
ah_df.render()

### final animation

There are many ways we can customize this like setting `inline_labels='Country/Region'` and chaining a `reference`.

In [ ]:
ah_df = ah.DataFrame(
    df_sort, 'Date', 'Cases', label='Country/Region',
    state_labels='Date', inline_labels='Country/Region',
    figsize=(10, 7), ylabel='Confirmed Cases / 100k People',
    join='cascade', ylim1s='explore_1',
    title='Confirmed COVID-19 Cases, normalized by population',
    note='Source: JHU CSSE COVID-19'
).reference(y0s='y', inline_labels='y')
ah_df.render()

That's about it for this tutorial. See the next tutorial to see how to create a bar chart race.

### full code

```python
import ahlive as ah
import pandas as pd

# load dataset
df = ah.tutorial.open_dataset('covid19_global_cases', raw=True, verbose=True)

# transform to tidy format
df_tidy = df.melt(
    id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'],
    var_name='Date', value_name='Cases')

# subset countries
countries = ['US', 'China', 'New Zealand', 'United Kingdom',
             'Brazil', 'India', 'Zambia', 'Pakistan']
df_subset = df_tidy.loc[df_tidy['Country/Region'].isin(countries)]

# find total cases per country instead of province/region
df_countries = df_subset.groupby(['Date', 'Country/Region'])[['Cases']].sum().reset_index()
df_countries = df_countries.loc[df_countries['Date'] >= '3/01/21']

# convert to datetime
df_dts = df_countries.copy()
df_dts['Date'] = pd.to_datetime(df_dts['Date'])
df_dts = df_dts.loc[df_dts['Date'] >= '2020-03-01']
df_dts = df_dts.groupby([
    pd.Grouper(key='Date', freq='7D'), 'Country/Region'
])['Cases'].last().reset_index()

# scale by a million
df_scale = df_dts.sort_values('Date')
df_scale['Cases'] /= 1e6

# normalize by population
df_pop = ah.tutorial.open_dataset('covid19_population')[['combined_key', 'population']]
df_norm = df_scale.merge(df_pop, left_on='Country/Region', right_on='combined_key')
df_norm['Cases'] = df_norm['Cases'] * 1e6 / df_norm['population']  # previously scaled by 1 million
df_norm['Cases'] *= 1e5  # scale by per 100k people

# sort by cases
ordering = df_norm.groupby('Country/Region').max().sort_values('Cases').index
df_sort = df_norm.copy()
df_sort['Country/Region'] = pd.Categorical(
    df_sort['Country/Region'], categories=ordering, ordered=True)
df_sort = df_sort.sort_values(['Country/Region', 'Date'])

# render
ah_df = ah.DataFrame(
    df_sort, 'Date', 'Cases', label='Country/Region',
    state_labels='Date', inline_labels='Country/Region',
    figsize=(10, 7), ylabel='Confirmed Cases / 100k People',
    join='cascade', ylim1s='explore_1',
    title='Confirmed COVID-19 Cases, normalized by population',
    note='Source: JHU CSSE COVID-19'
).reference(y0s='y', inline_labels='y')
ah_df.render()
```